### Cross Validation後にモデルを学習・保存する
### テストデータで、モデルの精度を確認する

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

import numpy as np
import random
from math import sqrt
import pandas as pd
import os, copy
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
from torch.nn.utils.rnn import pad_sequence

## データ準備

In [2]:
# ====== ユーザ設定 ======

excel_path = r"C:\Users\ryoya\MasterThesis\MT_Furuie\data\Miwa_LSTM_Data\CrossVal_5\Miwa_hourlyAve_for_LSTM_CrossVal_5.xlsx"
flood_idx_path_train = r"C:\Users\ryoya\MasterThesis\MT_Furuie\data\Miwa_LSTM_Data\CrossVal_5\Miwa_flood_idx_train.xlsx"
flood_idx_path_test = r"C:\Users\ryoya\MasterThesis\MT_Furuie\data\Miwa_LSTM_Data\CrossVal_5\Miwa_flood_idx_test.xlsx"


# 列番号（0始まり）で指定
enc_cols   = [5, 3, 4]      # エンコーダ入力の列番号（例：3変数）
dec_cols   = [5, 3]  # デコーダ入力の列番号（例：2変数）
y_cols      = [4]          # 出力（目的変数）の列番号（例：1変数）

Te = 72    # エンコーダのタイムステップ長
Td = 240   # デコーダのタイムステップ長


# 洪水区間（1始まり行番号で指定してOK。Python内部で0始まりに直す）
df_ranges_train = pd.read_excel(flood_idx_path_train, header=0)
flood_ranges_train_1based = [tuple(x) for x in df_ranges_train.to_numpy()]

# 0-basedに変換（pandasは0-based）
flood_ranges_train = [(s-1, e-1) for (s, e) in flood_ranges_train_1based]



# ====== 読み込み ======
df = pd.read_excel(excel_path, header=0)

# 必要列だけ抽出（順番固定）
use_cols = enc_cols + dec_cols + y_cols
data = df.iloc[:, use_cols].copy()


In [3]:
# ====== 標準化：train dataから平均・標準偏差を算出 ======

flood_data_train_parts = [data.iloc[s:e+1, :] for (s, e) in flood_ranges_train]
flood_data_train = pd.concat(flood_data_train_parts, axis=0)

mean = flood_data_train.mean(numeric_only=True)
std  = flood_data_train.std(numeric_only=True).replace(0, 1.0)
data_norm = (data - mean) / std # 全期間のデータを標準化

print('---平均（train）----')
print(mean)

print('----標準偏差（train）----')
print(std)

---平均（train）----
CumRain_24h     14.273778
Qin(m3/s)       46.917530
Tur(ppm)       351.369040
CumRain_24h     14.273778
Qin(m3/s)       46.917530
Tur(ppm)       351.369040
dtype: float64
----標準偏差（train）----
CumRain_24h     24.155857
Qin(m3/s)       47.874068
Tur(ppm)       933.516769
CumRain_24h     24.155857
Qin(m3/s)       47.874068
Tur(ppm)       933.516769
dtype: float64


In [4]:
class FloodSeq2SeqDataset(Dataset):
    def __init__(self, triplets):
        self.triplets = triplets  # list of (enc_X, dec_X, y)

    def __len__(self):
        return len(self.triplets)

    def __getitem__(self, idx):
        return self.triplets[idx]

def collate_fn(batch):
    enc_seqs, dec_seqs, ys = zip(*batch)
    # ここでは全サンプル同一長さ前提（Te/Td固定）なので単純stack
    enc_x = torch.stack(enc_seqs, dim=0)  # [B, Te, Fe]
    dec_x = torch.stack(dec_seqs, dim=0)  # [B, Td, Fd]
    y     = torch.stack(ys,       dim=0)  # [B, Td, Fo]
    
    # マスク（将来可変長のときに使う）
    B, Td, _ = y.shape
    mask = torch.ones(B, Td, 1, dtype=torch.float32)
    return enc_x, dec_x, y, mask
    

In [5]:
# ====== 洪水区間ごとにスライド窓でサンプル作成（train） ======

samples_train = []  # list of (enc_X: [Te, Fe], dec_X: [Td, Fd], y: [Td, Fo])
Fe, Fd, Fo = len(enc_cols), len(dec_cols), len(y_cols)

for (s, e) in flood_ranges_train:
    seg = data_norm.iloc[s:e+1]  # 区間データ（両端含む）
    n = len(seg)
    if n < Te + Td: # 1サンプルは Te + Tdの長さが必要
        continue
        
    for start in range(0, n - (Te + Td) + 1):
        enc_window = seg.iloc[start : start + Te]
        dec_window = seg.iloc[start + Te : start + Te + Td]
        # テンソル化
        enc_X = torch.tensor(enc_window.iloc[:, 0:Fe].to_numpy(dtype=np.float32))     # [Te, Fe]
        dec_X = torch.tensor(dec_window.iloc[:, Fe:Fe+Fd].to_numpy(dtype=np.float32))     # [Td, Fd]
        y     = torch.tensor(dec_window.iloc[:, Fe+Fd:Fe+Fd+Fo].to_numpy(dtype=np.float32))      # [Td, Fo]
        samples_train.append((enc_X, dec_X, y))


In [6]:
# ====== 洪水区間ごとにスライド窓でサンプル作成（test） ======

df_ranges_test = pd.read_excel(flood_idx_path_test, header=0)
flood_ranges_test_1based = [tuple(x) for x in df_ranges_test.to_numpy()]
flood_ranges_test = [(s-1, e-1) for (s, e) in flood_ranges_test_1based]


samples_test = []  # list of (enc_X: [Te, Fe], dec_X: [Td, Fd], y: [Td, Fo])
Fe, Fd, Fo = len(enc_cols), len(dec_cols), len(y_cols)

for (s, e) in flood_ranges_test:
    seg = data_norm.iloc[s:e+1]  # 区間データ（両端含む）
    n = len(seg)
    if n < Te + Td: # 1サンプルは Te + Tdの長さが必要
        continue
        
    for start in range(0, n - (Te + Td) + 1):
        enc_window = seg.iloc[start : start + Te]
        dec_window = seg.iloc[start + Te : start + Te + Td]
        # テンソル化
        enc_X = torch.tensor(enc_window.iloc[:, 0:Fe].to_numpy(dtype=np.float32))     # [Te, Fe]
        dec_X = torch.tensor(dec_window.iloc[:, Fe:Fe+Fd].to_numpy(dtype=np.float32))     # [Td, Fd]
        y     = torch.tensor(dec_window.iloc[:, Fe+Fd:Fe+Fd+Fo].to_numpy(dtype=np.float32))      # [Td, Fo]
        samples_test.append((enc_X, dec_X, y))


## 損失関数

In [7]:
def split_batch(batch):
    """collate_fnの戻り値が (enc,dec,y) か (enc,dec,y,mask) のどちらでも対応"""
    if len(batch) == 3:
        enc_x, dec_x, y = batch
        mask = torch.ones_like(y[..., :1])  # [B,Td,1]
    elif len(batch) == 4:
        enc_x, dec_x, y, mask = batch
    else:
        raise ValueError("Unexpected batch format")
    return enc_x.to(device), dec_x.to(device), y.to(device), mask.to(device)

def masked_mse(pred, target, mask):
    # pred/target: [B,T,Out], mask: [B,T,1] (1=valid, 0=pad)
    diff2 = (pred - target) ** 2
    diff2 = diff2 * mask
    denom = mask.sum(dim=(1,2)).clamp_min(1.0)  # per-sample
    per_sample = diff2.sum(dim=(1,2)) / denom
    return per_sample.mean()

def masked_rmse(pred, target, mask):
    return torch.sqrt(masked_mse(pred, target, mask))

def masked_r2(pred, target, mask):
    # R² = 1 - SSE/SST, マスク版
    mean = (target * mask).sum(dim=(1,2), keepdim=True) / mask.sum(dim=(1,2), keepdim=True).clamp_min(1.0)
    sse = ((pred - target) ** 2 * mask).sum(dim=(1,2))
    sst = ((target - mean) ** 2 * mask).sum(dim=(1,2)).clamp_min(1e-12)
    r2  = 1.0 - sse / sst
    return r2.mean()

def masked_corr(pred, target, mask):
    # pred/target: [B,T,Out], mask: [B,T,1]
    pred = pred * mask
    target = target * mask
    valid = mask.sum(dim=(1,2)).clamp_min(1.0)

    # 平均
    mean_pred = pred.sum(dim=(1,2)) / valid
    mean_target = target.sum(dim=(1,2)) / valid

    # 偏差
    diff_pred = (pred - mean_pred.view(-1,1,1)) * mask
    diff_target = (target - mean_target.view(-1,1,1)) * mask

    # 共分散と分散
    cov = (diff_pred * diff_target).sum(dim=(1,2)) / valid
    var_pred = (diff_pred**2).sum(dim=(1,2)) / valid
    var_target = (diff_target**2).sum(dim=(1,2)) / valid

    corr = cov / (torch.sqrt(var_pred * var_target) + 1e-12)
    return corr.mean()

## モデル定義（Seq2SeqLSTM）

In [8]:
class StateBridge(nn.Module):
    """
    Encoderの(h, c)をDecoder初期状態へ写像するブリッジ。
    - 層数/隠れ次元が異なってもOK
    - bridge_mode:
        - "zero_pad":  層合わせ=0埋め or 切り落とし、隠れ次元は線形射影
        - "repeat_top":層合わせ=最上層の繰り返し/切り落とし、隠れ次元は線形射影
        - "linear_stack": [B, L_enc, H_enc] -> 線形で [B, L_dec, H_dec] へ（層方向も学習で混合）

    - enc_layers: エンコーダの層の深さ
    - dec_layers: デコーダの層の深さ
    - enc_hidden: エンコーダのノード数
    - dec_hidden: デコーダのノード数
    """
    def __init__(self, enc_layers, dec_layers, enc_hidden, dec_hidden, mode="zero_pad"):
        super().__init__()
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.enc_hidden = enc_hidden
        self.dec_hidden = dec_hidden
        self.mode = mode

        # 隠れ次元の変換（h/c共用）
        if mode in ("zero_pad", "repeat_top"):
            self.proj = nn.Linear(enc_hidden, dec_hidden, bias=True)
        elif mode == "linear_stack":
            # 層方向もまとめて線形変換
            self.proj_h = nn.Linear(enc_layers * enc_hidden, dec_layers * dec_hidden, bias=True)
            self.proj_c = nn.Linear(enc_layers * enc_hidden, dec_layers * dec_hidden, bias=True)
        else:
            raise ValueError(f"Unknown bridge mode: {mode}")

    def _match_layers(self, x, how="zero_pad"):
        """
        x: [L_enc, B, H_enc] を層数だけ合わせる（隠れ次元は未変換）
        return: [L_dec, B, H_enc]

        B: バッチサイズ
        """
        L_enc, B, H = x.shape
        L_dec = self.dec_layers

        if L_dec == L_enc:
            return x

        if L_dec < L_enc:
            # 上位層を優先して切り落とす（直観的には最上層が一番抽象的）
            return x[:L_dec, :, :]

        # L_dec > L_enc の場合
        pad_count = L_dec - L_enc
        if how == "repeat_top":
            top = x[-1:, :, :].expand(pad_count, B, H)  # 最上層を複製
            return torch.cat([x, top], dim=0)
        else:  # zero_pad
            pad = x.new_zeros(pad_count, B, H)
            return torch.cat([x, pad], dim=0)

    def forward(self, h_enc, c_enc):
        """
        h_enc, c_enc: [L_enc, B, H_enc]
        返り値: (h0_dec, c0_dec) それぞれ [L_dec, B, H_dec]
        """
        if self.mode in ("zero_pad", "repeat_top"):
            # 層合わせ（まだ enc_hidden 次元のまま）
            h = self._match_layers(h_enc, "repeat_top" if self.mode=="repeat_top" else "zero_pad")
            c = self._match_layers(c_enc, "repeat_top" if self.mode=="repeat_top" else "zero_pad")
            # 次元射影
            L, B, H = h.shape
            h = self.proj(h)  # broadcasting: [L,B,H_enc]->[L,B,H_dec]
            c = self.proj(c)
            return h, c

        else:  # linear_stack
            # [L_enc,B,H_enc] -> [B, L_enc*H_enc]
            L_enc, B, H_enc = h_enc.shape
            flat_h = h_enc.transpose(0,1).reshape(B, L_enc*H_enc)
            flat_c = c_enc.transpose(0,1).reshape(B, L_enc*H_enc)
            # 線形写像
            out_h = self.proj_h(flat_h)  # [B, L_dec*H_dec]
            out_c = self.proj_c(flat_c)
            # [L_dec,B,H_dec] に戻す
            L_dec, H_dec = self.dec_layers, self.dec_hidden
            h = out_h.view(B, L_dec, H_dec).transpose(0,1).contiguous()
            c = out_c.view(B, L_dec, H_dec).transpose(0,1).contiguous()
            return h, c

In [9]:
class Seq2SeqLSTM(nn.Module):
    def __init__(
        self,
        in_enc: int, # エンコーダ入力変数の数
        in_dec: int, # デコーダ入力変数の数
        out_dim: int, # 出力変数の数
        enc_hidden: int = 128,
        dec_hidden: int = 128,
        enc_layers: int = 2,
        dec_layers: int = 3,
        bridge_mode: str = "zero_pad",  # "zero_pad" | "repeat_top" | "linear_stack"
        dropout: float = 0.0, # LSTMの層間ドロップアウト率
        bidirectional_enc: bool = False,  # エンコーダのみ双方向にするかどうか（必要ならEncoderを双方向にも）
        head_activation="relu", # 活性化関数
        use_conv: bool = True,
        conv_kernel: int = 3,           # 時間方向のカーネル幅（奇数推奨）
        conv_channels: int = None,      # 省略時は dec_hidden を維持
    ):
        super().__init__()
        self.bidirectional_enc = bidirectional_enc
        enc_dir = 2 if bidirectional_enc else 1
        enc_hidden_eff = enc_hidden * enc_dir  # 双方向なら出力次元が倍

        self.use_conv = bool(use_conv)

        self.enc = nn.LSTM(
            input_size=in_enc,
            hidden_size=enc_hidden,
            num_layers=enc_layers,
            batch_first=True,
            dropout=dropout if enc_layers > 1 else 0.0,
            bidirectional=bidirectional_enc
        )

        self.dec = nn.LSTM(
            input_size=in_dec,
            hidden_size=dec_hidden,
            num_layers=dec_layers,
            batch_first=True,
            dropout=dropout if dec_layers > 1 else 0.0
        )

        # Encoderが双方向のときは、(h_fwd, h_bwd) を結合した次元 enc_hidden_eff を
        # Decoder hidden 次元へ写像する必要がある
        self.bridge = StateBridge(
            enc_layers=enc_layers * enc_dir,
            dec_layers=dec_layers,
            enc_hidden=enc_hidden,
            dec_hidden=dec_hidden,
            mode=bridge_mode
        ) if (enc_layers != dec_layers or enc_dir != 1 or enc_hidden != dec_hidden or bridge_mode=="linear_stack") else None

        
        # ---------- 中間層（1次元畳み込み） ----------
        if conv_channels is None:
            conv_channels = dec_hidden
        self.conv_channels = conv_channels
        if self.use_conv:
            # Conv1dは [B, C=特徴, T=時間] を受け取るので後で転置する
            padding = conv_kernel // 2  # SAME相当（奇数カーネル推奨）
            self.conv1d = nn.Conv1d(
                in_channels=dec_hidden,
                out_channels=conv_channels,
                kernel_size=conv_kernel,
                padding=padding
            )
            self.conv_act = {
                "identity": nn.Identity(),
                "relu": nn.ReLU(),
                "tanh": nn.Tanh(),
                "sigmoid": nn.Sigmoid(),
            }.get(head_activation, nn.ReLU())
            self.conv_dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()

            # 出力ヘッドは conv_channels → out_dim
            self.head = nn.Linear(conv_channels, out_dim)
            # ここでの self.act はヘッド直前では使わない（Conv後に適用済み）
            self.act = nn.Identity()
        else:
            # 畳み込みを使わない場合は dec_hidden → out_dim
            self.head = nn.Linear(dec_hidden, out_dim)
            self.act = {
                "identity": nn.Identity(),
                "relu": nn.ReLU(),
                "tanh": nn.Tanh(),
                "sigmoid": nn.Sigmoid(),
            }[head_activation]

    def _extract_final_states(self, out, hc):
        """
        LSTMの出力から (h_T, c_T) を取り出して成形。
        双方向Encoderの場合は各層ごとに [fwd, bwd] を層方向に並べる。
        """
        h, c = hc  # [num_layers * num_directions, B, H]
        return h, c

    def forward(self, enc_x, dec_x):
        """
        enc_x: [B, Te, in_enc]
        dec_x: [B, Td, in_dec]
        return: yhat [B, Td, out_dim]
        """
        # ----- Encoder -----
        _, (h_T, c_T) = self.enc(enc_x)  # h_T,c_T: [L_enc * dir, B, H_enc]

        # ----- Bridge -----
        if self.bridge is not None:
            h0_dec, c0_dec = self.bridge(h_T, c_T)  # [L_dec, B, H_dec]
        else:
            h0_dec, c0_dec = h_T, c_T

        # ----- Decoder -----
        dec_out, _ = self.dec(dec_x, (h0_dec, c0_dec))  # [B, Td, H_dec]

        if self.use_conv:
            # 時間方向のConv1d: [B, H_dec, Td] -> Conv -> [B, C, Td] -> [B, Td, C]
            x = dec_out.transpose(1, 2)             # [B, H_dec, Td]
            x = self.conv1d(x)                      # [B, conv_channels, Td]
            x = self.conv_act(x)
            x = self.conv_dropout(x)
            x = x.transpose(1, 2)                   # [B, Td, conv_channels]
            yhat = self.head(x)                     # [B, Td, out_dim]
        else:
            # 従来パス：活性化→線形
            yhat = self.head(self.act(dec_out))     # [B, Td, out_dim]

        return yhat
        

In [10]:
# 重み初期化用の関数

def init_weights(m):
    if isinstance(m, nn.Linear):
        # 例: Xavier（正規分布）
        # init.xavier_normal_(m.weight)
        # ReLU系のLinearならHeにしたい場合:
        init.kaiming_normal_(m.weight, nonlinearity="relu")
        if m.bias is not None:
            init.zeros_(m.bias)

    elif isinstance(m, nn.Conv1d):
        # Conv1d も Linear と同様に初期化可能
        # 例: Xavier（正規分布）
        # init.xavier_normal_(m.weight)
        # ReLU系のLinearならHeにしたい場合:
        init.kaiming_normal_(m.weight, nonlinearity="relu")
        if m.bias is not None:
            init.zeros_(m.bias)

    elif isinstance(m, nn.LSTM):
        # LSTM の場合は named_parameters() で内部ゲートを個別に初期化
        for name, param in m.named_parameters():
            if "weight_ih" in name:
                # 入力→隠れ：Xavier が安定
                with torch.no_grad():
                    init.xavier_uniform_(param)
            elif "weight_hh" in name:
                # 隠れ→隠れ：Orthogonal が定番
                with torch.no_grad():
                    init.orthogonal_(param)
            elif "bias" in name:
                with torch.no_grad():
                    init.zeros_(param)
                    # 必要なら忘却ゲートバイアスを +1 初期化することも可能
                    # hidden_size = param.shape[0] // 4
                    # param[hidden_size:2*hidden_size] = 1.0


## ハイパーパラメータの設定
### ※保存Excelファイルのパスも設定

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cfg = {
    # Model
    "in_enc": 3,
    "in_dec": 2,
    "out_dim": 1,
    "enc_hidden": 16, # 【要変更】
    "dec_hidden": 16, # 【要変更】
    "enc_layers": 1, # 【要変更】
    "dec_layers": 1, # 【要変更】
    "bridge_mode": "zero_pad",   # "zero_pad" | "repeat_top" | "linear_stack"
    "batch_size": 256, # 【要変更】
    "dropout": 0.1,
    "bidirectional_enc": False,
    "head_activation": "relu", # "identity", "relu", "tanh", "sigmoid" など 【要変更】
    # Train
    "epochs": 100,
    "lr": 1e-2,
    "weight_decay": 0.0, # L2正規化係数。0なら無効
    "grad_clip": 1.0, # 勾配クリッピングの閾値。０かNoneなら無効
    "print_every": 1, # 学習の進捗を何エポックごとに出力するか
    "patience": 5, # early stopping
    "use_conv": True # 畳み込み層を使うかどうか【要変更】
}

save_dir = r"C:\Users\ryoya\MasterThesis\MT_Furuie\results\Miwa_LSTM\CrossVal_5\model"
save_path_excel = r"C:\Users\ryoya\MasterThesis\MT_Furuie\results\Miwa_LSTM\CrossVal_5\model\CrossVal_5_result_final.xlsx" # 【要変更】
# save_path_model = r"C:\Users\ryoya\MasterThesis\MT_Furuie\results\Miwa_LSTM\CrossVal_2/CrossVal_2_result_final_model"


## 学習関数・評価関数の定義

In [12]:
# 学習関数・評価関数の定義

def train_one_epoch(model, loader, optimizer):
    model.train()
    total_loss = 0.0
    n = 0
    for batch in loader:
        enc_x, dec_x, y, mask = split_batch(batch)
        yhat = model(enc_x, dec_x)
        loss = masked_mse(yhat, y, mask)

        optimizer.zero_grad()
        loss.backward()
        if cfg["grad_clip"]:
            nn.utils.clip_grad_norm_(model.parameters(), cfg["grad_clip"])
        optimizer.step()

        bs = enc_x.size(0) # バッチサイズ
        total_loss += loss.item() * bs
        n += bs
    return total_loss / max(n,1)

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    total_loss = 0.0; total_rmse = 0.0; total_r2 = 0.0; total_corr = 0.0; n = 0
    for batch in loader:
        enc_x, dec_x, y, mask = split_batch(batch)
        yhat = model(enc_x, dec_x)

        loss = masked_mse(yhat, y, mask)
        rmse = masked_rmse(yhat, y, mask)
        r2   = masked_r2(yhat, y, mask)
        corr = masked_corr(yhat, y, mask)

        bs = enc_x.size(0)
        total_loss += loss.item() * bs
        total_rmse += rmse.item() * bs
        total_r2   += r2.item() * bs
        total_corr += corr.item() * bs
        n += bs

    return {
        "loss": total_loss / max(n,1),
        "rmse": total_rmse / max(n,1),
        "r2":   total_r2   / max(n,1),
        "corr": total_corr / max(n,1),
    }

In [13]:
def inverse_standardize_y_by_index(y_std, mean, std, y_pos):
    """
    y_std:  標準化スケールの出力テンソル [B, T, Fo]
    mean, std: pandas.Series（学習時にfitしたもの。index=列名）
    y_pos: 出力列の「列番号（位置）」リスト（例: [Fe+Fd, Fe+Fd+1, ...]）
    return: 元スケールの y [B, T, Fo]
    """
    m = torch.tensor(mean.iloc[y_pos].to_numpy(dtype=float),
                     dtype=y_std.dtype, device=y_std.device).view(1, 1, -1)
    s = torch.tensor(std.iloc[y_pos].to_numpy(dtype=float),
                     dtype=y_std.dtype, device=y_std.device).view(1, 1, -1)
    return y_std * s + m


def masked_corr(pred, target, mask, eps: float = 1e-12):
    """
    マスク付きピアソン相関係数（バッチ平均）
    pred/target: [B, T, Fo], mask: [B, T, 1]（1=有効, 0=無効）
    返り値: スカラー（バッチ平均の相関）
    """
    # 有効点数（サンプルごと）
    valid = mask.sum(dim=(1, 2)).clamp_min(1.0)  # [B]

    # 平均（サンプルごと）
    mean_p = (pred * mask).sum(dim=(1, 2), keepdim=True) / valid.view(-1, 1, 1)
    mean_t = (target * mask).sum(dim=(1, 2), keepdim=True) / valid.view(-1, 1, 1)

    # 偏差
    dp = (pred - mean_p) * mask
    dt = (target - mean_t) * mask

    # 共分散・分散（サンプルごと）
    cov = dp.mul(dt).sum(dim=(1, 2)) / valid        # [B]
    var_p = dp.pow(2).sum(dim=(1, 2)) / valid       # [B]
    var_t = dt.pow(2).sum(dim=(1, 2)) / valid       # [B]

    corr = cov / (torch.sqrt(var_p * var_t) + eps)  # [B]
    return corr.mean()


@torch.no_grad()
def evaluate_original_scale_by_index(model, loader, mean, std, data_columns, y_pos):
    model.eval()
    device = next(model.parameters()).device

    # 厳密集計用
    total_sse = 0.0   # 全有効点での誤差二乗和
    total_cnt = 0.0   # 全有効点数（マスク=1の総数）
    total_r2  = 0.0   # R² のバッチ加重平均用
    total_corr = 0.0  # 相関R のバッチ加重平均用
    n = 0             # サンプル数（バッチ内のBの合計）

    for batch in loader:
        if len(batch) == 3:
            enc_x, dec_x, y_std = batch
            mask = torch.ones_like(y_std[..., :1])
        elif len(batch) == 4:
            enc_x, dec_x, y_std, mask = batch
        else:
            raise ValueError("Unexpected batch format")

        # 統一デバイス・dtype
        enc_x = enc_x.to(device=device, dtype=torch.float32)
        dec_x = dec_x.to(device=device, dtype=torch.float32)
        y_std = y_std.to(device=device, dtype=torch.float32)
        mask  = mask.to(device=device, dtype=torch.float32)

        # 予測（標準化スケール）
        yhat_std = model(enc_x, dec_x)

        # 出力だけ逆標準化
        yhat = inverse_standardize_y_by_index(yhat_std, mean, std, y_pos)
        y    = inverse_standardize_y_by_index(y_std,     mean, std, y_pos)

        # 厳密RMSE用：SSEと有効点数を直接合算
        sse_batch = ((yhat - y) ** 2 * mask).sum().item()
        cnt_batch = mask.sum().item()
        total_sse += sse_batch
        total_cnt += max(cnt_batch, 1.0)

        # R² と 相関R はサンプル数で加重平均
        r2    = masked_r2(yhat, y, mask).item()
        corr  = masked_corr(yhat, y, mask).item()
        bs = enc_x.size(0)
        total_r2   += r2   * bs
        total_corr += corr * bs
        n += bs

    mse  = total_sse / max(total_cnt, 1.0)
    rmse = mse ** 0.5
    r2   = total_r2   / max(n, 1)
    corr = total_corr / max(n, 1)

    return {"mse": mse, "rmse": rmse, "r2": r2, "corr": corr}

## 学習（シード値を変えて5回学習。結果を保存する）
### ※train期間のうちランダムに85%を学習、15%を検証に分割する

In [14]:
seeds = [0, 1, 2, 3, 4]               # モデルの初期値を決める際のシード値
y_pos = [Fe+Fd]
global_best_val = float("inf")

rmse_list = np.zeros((5, 2))
r2_list = np.zeros((5, 2))

for s_id, seed in enumerate(seeds):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    # DataLoaderの作成
    random.shuffle(samples_train) # ランダムに並び替え
    train_val_Dataset = FloodSeq2SeqDataset(samples_train)

    n_total = len(train_val_Dataset)
    n_train = int(n_total*0.85)
    n_val = n_total - n_train
    trainDataset, valDataset = random_split(train_val_Dataset, [n_train, n_val])

    trainLoader = DataLoader(trainDataset, batch_size=cfg["batch_size"], shuffle=True, collate_fn=collate_fn)
    valLoader = DataLoader(valDataset, batch_size=cfg["batch_size"], shuffle=False, collate_fn=collate_fn)

    
    # モデルの初期化
    model = Seq2SeqLSTM(in_enc=cfg["in_enc"], in_dec=cfg["in_dec"], out_dim=cfg["out_dim"],
                        enc_hidden=cfg["enc_hidden"], dec_hidden=cfg["dec_hidden"],
                        enc_layers=cfg["enc_layers"], dec_layers=cfg["dec_layers"],
                        bridge_mode=cfg["bridge_mode"], dropout=cfg["dropout"],
                        bidirectional_enc=cfg["bidirectional_enc"],
                        head_activation=cfg["head_activation"],
                        use_conv=cfg["use_conv"]
                       ).to(device)
    optimizer = optim.Adam(model.parameters(), lr=cfg["lr"], weight_decay=cfg["weight_decay"])

    model.apply(init_weights)


    best_val = float("inf")
    best_epoch = 0
    epochs_no_improve = 0
    best_model_state = None
    best_opt_state = None
    best_val_metrics = None

    patience = cfg["patience"]
    min_delta = 1e-4

    # 学習ループ（early stoppingあり）
    for epoch in range(1, cfg["epochs"] + 1):
        # ---- 1. 学習 ----
        train_loss = train_one_epoch(model, trainLoader, optimizer)

        # ---- 2. 検証 ----
        val_metrics = evaluate(model, valLoader)
        val_loss = float(val_metrics["loss"])

        # ---- 3. ログ出力 ----
        if epoch % cfg["print_every"] == 0:
            print(f"[{epoch}/{cfg['epochs']}] "
                  f"train_loss={train_loss:.4f} | "
                  f"val_loss={val_loss:.4f} "
                  f"val_rmse={val_metrics['rmse']:.4f} "
                  f"val_r2={val_metrics['r2']:.4f}"
                  f"val_corr={val_metrics['corr']:.4f}")

        # ---- 4. 改善チェック ----
        if best_val - val_loss > min_delta:
            best_val = val_loss
            best_epoch = epoch
            epochs_no_improve = 0
            
            # ★ モデル重みをcloneして保持
            best_model_state = {k: v.detach().clone() for k, v in model.state_dict().items()}
            # ★ Optimizerの状態もcloneして保持（必要に応じて）
            best_opt_state = {
                "state": {
                    k: {kk: (vv.detach().clone() if torch.is_tensor(vv) else vv)
                        for kk, vv in v.items()}
                    for k, v in optimizer.state_dict()["state"].items()
                },
                "param_groups": [dict(g) for g in optimizer.state_dict()["param_groups"]],
            }
    
        else:
            epochs_no_improve += 1

        # ---- 5. Early Stopping 発動 ----
        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {epoch} "
                  f"(best epoch={best_epoch}, val_loss={best_val:.4f})")
            break

    assert best_model_state is not None, "best_model_state が None です。学習が行われていない可能性があります。"
    model.load_state_dict(best_model_state)


    # 学習終了後に、実スケールでの標準化パラメータを記録
    metrics = evaluate_original_scale_by_index(model, valLoader, mean, std, data.columns, y_pos)
    metrics_train = evaluate_original_scale_by_index(model, trainLoader, mean, std, data.columns, y_pos)
    
    rmse_list[s_id, 1] = metrics["rmse"]
    r2_list[s_id, 1] = metrics["r2"]
    rmse_list[s_id, 0] = metrics_train["rmse"]
    r2_list[s_id, 0] = metrics_train["r2"]

    ckpt_name = f"seed{seed}"
    save_path_temp = os.path.join(save_dir, ckpt_name)

    torch.save({
        "model_state": best_model_state,          # ★ベスト重み
        "optimizer_state": best_opt_state,        # 再開したい場合に使用
        "best_epoch": best_epoch,
        "best_val_loss": best_val,
        "best_val_metrics": best_val_metrics,     # スケール前の検証指標（任意）
        "val_metrics_original_scale": metrics,    # 実スケール評価
        "cfg": cfg,
        "seed": seed,
        "scaler": {
            "mean": mean,                        # pandas.Series を含むなら必要に応じて .to_dict() でもOK
            "std":  std,
            "data_columns": list(data.columns),
            "y_pos": y_pos,
        }
    }, save_path_temp)

    print(f"{s_id+1} 回目の学習が終了しました。ベストモデルを保存: {save_path_temp}")

    if best_val < global_best_val:
        global_best_val = best_val
        global_best_info = (seed, save_path_temp, best_epoch, best_val)


print("すべての学習が終了しました。")

if global_best_info is not None:
    g_seed, g_path, g_epoch, g_loss = global_best_info
    print(f"【全seedで最良】seed={g_seed}, best_epoch={g_epoch}, val_loss={g_loss:.6f}")
    print(f"→ パス: {g_path}")


[1/100] train_loss=0.9904 | val_loss=0.8325 val_rmse=0.9123 val_r2=-45.6097val_corr=0.6684
[2/100] train_loss=0.6262 | val_loss=0.5065 val_rmse=0.7116 val_r2=-10.4074val_corr=0.7614
[3/100] train_loss=0.4087 | val_loss=0.3770 val_rmse=0.6140 val_r2=-13.5184val_corr=0.7655
[4/100] train_loss=0.3454 | val_loss=0.3278 val_rmse=0.5725 val_r2=-11.3045val_corr=0.7614
[5/100] train_loss=0.3094 | val_loss=0.2789 val_rmse=0.5281 val_r2=-6.2643val_corr=0.7811
[6/100] train_loss=0.2708 | val_loss=0.2391 val_rmse=0.4890 val_r2=-8.9562val_corr=0.7896
[7/100] train_loss=0.2435 | val_loss=0.2159 val_rmse=0.4647 val_r2=-13.7078val_corr=0.7954
[8/100] train_loss=0.2279 | val_loss=0.2007 val_rmse=0.4480 val_r2=-11.1973val_corr=0.8025
[9/100] train_loss=0.2115 | val_loss=0.1819 val_rmse=0.4265 val_r2=-8.0405val_corr=0.8102
[10/100] train_loss=0.2003 | val_loss=0.1710 val_rmse=0.4135 val_r2=-8.5039val_corr=0.8191
[11/100] train_loss=0.1858 | val_loss=0.1582 val_rmse=0.3977 val_r2=-7.7079val_corr=0.8191
[1

In [15]:
# 結果をエクセルに保存
print(rmse_list)
print(r2_list)

with pd.ExcelWriter(save_path_excel, engine="openpyxl") as writer:
    pd.DataFrame(rmse_list).to_excel(writer, sheet_name="rmse_list", index=False, header=False)
    pd.DataFrame(r2_list).to_excel(writer, sheet_name="r2_list", index=False, header=False)

print("結果をExcelファイルに保存しました。")


[[234.60721622 243.36134737]
 [192.23022828 187.35264578]
 [155.46366138 151.8377075 ]
 [163.23028469 165.54694951]
 [236.37133092 240.46161806]]
[[ -4.57521478 -11.58328152]
 [ -1.8313336   -2.77354991]
 [ -0.10170799   0.19234409]
 [ -0.47808661  -0.28804847]
 [ -6.01550584  -7.99945715]]
結果をExcelファイルに保存しました。


## 最も良いモデルをロードし、テストデータで評価する

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 最も良かったモデルのパスを指定
best_model_path = r"C:\Users\ryoya\MasterThesis\MT_Furuie\results\Miwa_LSTM\CrossVal_5\model\seed2"

ckpt = torch.load(best_model_path, map_location="cpu")

# モデルの再構築
cfg = ckpt["cfg"]
model = Seq2SeqLSTM(in_enc=cfg["in_enc"], in_dec=cfg["in_dec"], out_dim=cfg["out_dim"],
                    enc_hidden=cfg["enc_hidden"], dec_hidden=cfg["dec_hidden"],
                    enc_layers=cfg["enc_layers"], dec_layers=cfg["dec_layers"],
                    bridge_mode=cfg["bridge_mode"], dropout=cfg["dropout"],
                    bidirectional_enc=cfg["bidirectional_enc"],
                    head_activation=cfg["head_activation"],
                    use_conv=cfg["use_conv"]
                   ).to(device)

# パラメータをロード
model.load_state_dict(ckpt["model_state"])
model.eval() # 評価モードに切り替え

# 標準化パラメータの取り出し
scaler = ckpt["scaler"]
mean = scaler["mean"]                # pandas.Series
std  = scaler["std"]                 # pandas.Series
data_columns = scaler["data_columns"]  # 列名リスト
y_pos = scaler["y_pos"]


C:\Users\ryoya\AppData\Local\Temp\ipykernel_17284\3204996539.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(best_model_path, map_location="cpu")


## testLoaderを作成し、モデルの精度を評価

In [17]:

testDataset = FloodSeq2SeqDataset(samples_test)
testLoader = DataLoader(testDataset, batch_size=cfg["batch_size"], shuffle=False, collate_fn=collate_fn)

metrics_test = evaluate_original_scale_by_index(model, testLoader, mean, std, data_columns, y_pos)

print(metrics_test)

test_save_path = r"C:\Users\ryoya\MasterThesis\MT_Furuie\results\Miwa_LSTM\CrossVal_5\model\CrossVal_5_result_test.xlsx"

# 辞書をDataFrameに変換
df_metrics_test = pd.DataFrame([metrics_test])   # []で囲むと1行の表になる

# Excelに出力
df_metrics_test.to_excel(test_save_path, index=False)


{'mse': 465374.8566146736, 'rmse': 682.1838876832797, 'r2': -15.647573585423482, 'corr': 0.6554581143827124}


## 任意の時刻における予測をエクセルファイルに出力

In [24]:
# 出力したい部分の開始行番号を指定（0はじまり）
s = 56400
pred_save_path = r"C:\Users\ryoya\MasterThesis\MT_Furuie\results\Miwa_LSTM\CrossVal_5\model\CrossVal_5_pred_2025_06_08.xlsx"

# エンコーダ開始時刻（予測期間は3日後から）
# (4320: 2019_06_30), (38664: 2023_05_31), (38760: 2024_06_04), (13032: 2020_06_27), (13176: 2020_07_03), (48048: 2024_06_25), (56400: 2025_06_08)


Fe, Fd, Fo = cfg["in_enc"], cfg["in_dec"], cfg["out_dim"]

enc_window = data_norm.iloc[s : s + Te]
dec_window = data_norm.iloc[s + Te : s + Te + Td]
# テンソル化
enc_X = torch.tensor(enc_window.iloc[:, 0:Fe].to_numpy(dtype=np.float32))
dec_X = torch.tensor(dec_window.iloc[:, Fe:Fe+Fd].to_numpy(dtype=np.float32))
y     = torch.tensor(dec_window.iloc[:, Fe+Fd:Fe+Fd+Fo].to_numpy(dtype=np.float32))

# モデルに入力できる形へ変形
enc_in = enc_X.unsqueeze(0).to(device)
dec_in = dec_X.unsqueeze(0).to(device)

# デコーダに対応する時刻、実測のYを抽出
dec_T = df.iloc[s+Te:s+Te+Td, 0:2]
obs_Y = df.iloc[s+Te:s+Te+Td, y_cols]

# モデルから予測
pred_Y_norm = model(enc_in, dec_in)
pred_Y_norm = pred_Y_norm.squeeze(0)

# 逆標準化
ymean = mean.iloc[y_pos]
ystd = std.iloc[y_pos]
ymean_t = torch.tensor(ymean.values, dtype=torch.float32, device=pred_Y_norm.device).view(1, -1)
ystd_t  = torch.tensor(ystd.values,  dtype=torch.float32, device=pred_Y_norm.device).view(1, -1)
pred_Y = pred_Y_norm * ystd_t + ymean_t


# 1. pandas Series → DataFrame
dec_T_df = pd.DataFrame(dec_T).reset_index(drop=True)
obs_Y_df  = pd.DataFrame(obs_Y).reset_index(drop=True)

# 2. Tensor → NumPy → DataFrame
pred_Y_df = pd.DataFrame(pred_Y.detach().cpu().numpy())
dec_X_df  = pd.DataFrame(dec_X.detach().cpu().numpy())

# 3. 横方向に結合
out_df = pd.concat([dec_T_df, obs_Y_df, pred_Y_df, dec_X_df], axis=1)


out_df.to_excel(pred_save_path, index=False)

print("Excelに保存しました:", pred_save_path)


Excelに保存しました: C:\Users\ryoya\MasterThesis\MT_Furuie\results\Miwa_LSTM\CrossVal_5\model\CrossVal_5_pred_2025_06_08.xlsx
